### CREACIÓN Y CONEXIÓN A BASE DE DATOS (AMAZON AWS)

CREACIÓN: Se ha levantado una instancia de BD de MySQL en Amazon AWS (RDS). Esto hace que la base de datos sea facilmente escalable y añade una capa extra de seguridad. 

En este caso, se ha confugirado la conectividad para que la BBDD sea accesible desde cualquier dirección I.P para asegurar que todo el codigo es replicable en cualquier ordenador de Grenergy.

In [5]:
# pip install pymysql

In [49]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

In [21]:
#Datos de la instancia para ejecutar la conexión

username = "admin"
password = "GrenergyData"
host = "database-1.cz08iukiw4br.eu-north-1.rds.amazonaws.com" 
port = 3306

In [16]:
'''
pymysql.cursors.DictCursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplEs. Asi podemos acceder por clave a las columnas.
'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
)

# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

In [22]:
'''
Este es el engine version de la BD de AWS
'''

cursor.execute('SELECT VERSION()')
version = cursor.fetchone()
print(f'MySQL version: {version}')

MySQL version: {'VERSION()': '8.0.35'}


In [18]:
# Creamos una BD con lenguaje SQL. Tenemos una instancia de MySQL, pero no una BD
create_db = '''CREATE DATABASE data_mediciones_chile'''
cursor.execute(create_db)

1

In [23]:
#comprobamos nuestras BBDD
cursor.execute('SHOW DATABASES')
cursor.fetchall()

[{'Database': 'data_mediciones_chile'},
 {'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'}]

In [24]:
# Para usar la BD  recien creada

cursor.connection.commit()
use_db = ''' USE data_mediciones_chile'''
cursor.execute(use_db)

0

In [25]:
# crear una tabla que se llame CEN con los campos contenidos en el Informe Excel previamente realizado (Coordinador Electrico Nacional de CHile)

create_table = '''
CREATE TABLE CEN (
    id INT NOT NULL auto_increment,
    year INT,
    mes INT,
    dia INT,
    hora INT,
    minuto INT,
    CURICO_013_PMGD7_LCH DOUBLE,
    MALLOCO_012_PMGD5_PNF DOUBLE,
    RENGO_015_PMGD5_SLM DOUBLE,
    TAMANGO_066_B1_IUN DOUBLE,
    URIBE_023_PMGD2_UAS DOUBLE,
    primary key (id)
)
'''
cursor.execute(create_table)

0

In [51]:
# checkear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_data_mediciones_chile': 'CEN'}]

#### Insertar datos del informe Excel

In [33]:
df = pd.read_excel('Informe_mediciones_puntos.xlsx',sheet_name='mediciones raw')

In [38]:
df.head(10)

,year,mes,dia,hora,minuto,CURICO_013_PMGD7_LCH,MALLOCO_012_PMGD5_PNF,RENGO_015_PMGD5_SLM,TAMANGO_066_B1_IUN,URIBE_023_PMGD2_UAS
0,2024,5,1,0,0,0.0,0.0,0.0,0.0,0.0
1,2024,5,1,0,15,0.0,0.0,0.0,0.0,0.0
2,2024,5,1,0,30,0.0,0.0,0.0,0.0,0.0
3,2024,5,1,0,45,0.0,0.0,0.0,0.0,0.0
4,2024,5,1,1,0,0.0,0.0,0.0,0.0,0.0
5,2024,5,1,1,15,0.0,0.0,0.0,0.0,0.0
6,2024,5,1,1,30,0.0,0.0,0.0,0.0,0.0
7,2024,5,1,1,45,0.0,0.0,0.0,0.0,0.0
8,2024,5,1,2,0,0.0,0.0,0.0,0.0,0.0
9,2024,5,1,2,15,0.0,0.0,0.0,0.0,0.0


Aunque el DF ya está en orden de mas antiguo a mas reciente, volvemos a ordenarlo para tenerlo listo para futuros datos a insertar

In [47]:
#Antiguo a reciente

df_sorted = df.sort_values(by=['year', 'mes', 'dia', 'hora', 'minuto'])

In [48]:
df_sorted

,year,mes,dia,hora,minuto,CURICO_013_PMGD7_LCH,MALLOCO_012_PMGD5_PNF,RENGO_015_PMGD5_SLM,TAMANGO_066_B1_IUN,URIBE_023_PMGD2_UAS
0,2024,5,1,0,0,0.0,0.0,0.0,0.000000,0.0
1,2024,5,1,0,15,0.0,0.0,0.0,0.000000,0.0
2,2024,5,1,0,30,0.0,0.0,0.0,0.000000,0.0
3,2024,5,1,0,45,0.0,0.0,0.0,0.000000,0.0
4,2024,5,1,1,0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
8827,2024,7,31,22,45,0.0,0.0,0.0,17.907053,0.0
8828,2024,7,31,23,0,0.0,0.0,0.0,17.967394,0.0
8829,2024,7,31,23,15,0.0,0.0,0.0,17.361639,0.0
8830,2024,7,31,23,30,0.0,0.0,0.0,17.773249,0.0


In [53]:
#Creamos engine para insertar datos con sqlalchemy

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = 'data_mediciones_chile'))

In [54]:
# insertamos todo el dataframe, si la tabla existe, añade los registros despues de los anteriores. Si la tabla no existe previamente, la crea.

df.to_sql(name='CEN', con=engine, if_exists= 'append', index=False)

8832

In [55]:
#Para guardar los cambios

db.commit()

In [56]:
# leemos los datos para comprobar que se han ingestado correctamente
sql = '''SELECT * FROM CEN'''
cursor.execute(sql)
mi_tabla = cursor.fetchall()
mi_tabla

[{'id': 1,
  'year': 2024,
  'mes': 5,
  'dia': 1,
  'hora': 0,
  'minuto': 0,
  'CURICO_013_PMGD7_LCH': 0.0,
  'MALLOCO_012_PMGD5_PNF': 0.0,
  'RENGO_015_PMGD5_SLM': 0.0,
  'TAMANGO_066_B1_IUN': 0.0,
  'URIBE_023_PMGD2_UAS': 0.0},
 {'id': 2,
  'year': 2024,
  'mes': 5,
  'dia': 1,
  'hora': 0,
  'minuto': 15,
  'CURICO_013_PMGD7_LCH': 0.0,
  'MALLOCO_012_PMGD5_PNF': 0.0,
  'RENGO_015_PMGD5_SLM': 0.0,
  'TAMANGO_066_B1_IUN': 0.0,
  'URIBE_023_PMGD2_UAS': 0.0},
 {'id': 3,
  'year': 2024,
  'mes': 5,
  'dia': 1,
  'hora': 0,
  'minuto': 30,
  'CURICO_013_PMGD7_LCH': 0.0,
  'MALLOCO_012_PMGD5_PNF': 0.0,
  'RENGO_015_PMGD5_SLM': 0.0,
  'TAMANGO_066_B1_IUN': 0.0,
  'URIBE_023_PMGD2_UAS': 0.0},
 {'id': 4,
  'year': 2024,
  'mes': 5,
  'dia': 1,
  'hora': 0,
  'minuto': 45,
  'CURICO_013_PMGD7_LCH': 0.0,
  'MALLOCO_012_PMGD5_PNF': 0.0,
  'RENGO_015_PMGD5_SLM': 0.0,
  'TAMANGO_066_B1_IUN': 0.0,
  'URIBE_023_PMGD2_UAS': 0.0},
 {'id': 5,
  'year': 2024,
  'mes': 5,
  'dia': 1,
  'hora': 1,
  'mi

In [57]:
#Comprobación de que la tabla se ha cargado correctamente

pd.DataFrame(mi_tabla)

,id,year,mes,dia,hora,minuto,CURICO_013_PMGD7_LCH,MALLOCO_012_PMGD5_PNF,RENGO_015_PMGD5_SLM,TAMANGO_066_B1_IUN,URIBE_023_PMGD2_UAS
0,1,2024,5,1,0,0,0.0,0.0,0.0,0.000000,0.0
1,2,2024,5,1,0,15,0.0,0.0,0.0,0.000000,0.0
2,3,2024,5,1,0,30,0.0,0.0,0.0,0.000000,0.0
3,4,2024,5,1,0,45,0.0,0.0,0.0,0.000000,0.0
4,5,2024,5,1,1,0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
8827,8828,2024,7,31,22,45,0.0,0.0,0.0,17.907053,0.0
8828,8829,2024,7,31,23,0,0.0,0.0,0.0,17.967394,0.0
8829,8830,2024,7,31,23,15,0.0,0.0,0.0,17.361639,0.0
8830,8831,2024,7,31,23,30,0.0,0.0,0.0,17.773249,0.0


In [58]:
# Cerrar conexión mysql
db.close()

# Cerrar conexión sqlalchemy
engine.dispose()